<a href="https://colab.research.google.com/github/mikeroguez/experimento/blob/main/Merge_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importar bibliotecas necesarias
import pandas as pd

# Cargar los datasets desde archivos locales
assignments_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas_consolidado_cleaned_sin_cursos_reutilizados.csv"
exams_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/examenes/examenes_consolidados.csv"
access_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/accesos/accesos_consolidado_cleaned.csv"
forums_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/foros/foros_consolidado.csv"

assignments_df = pd.read_csv(assignments_path)
exams_df = pd.read_csv(exams_path)
access_df = pd.read_csv(access_path)
forums_df = pd.read_csv(forums_path)

# 1. Integrar "Exams" a "Assignments" basado en groupKey y email
merged_df = pd.merge(assignments_df, exams_df, how="left", on=["groupKey", "email"])

# 2. Integrar "Access" a la base intermedia usando curso, groupKey y email
merged_df = pd.merge(merged_df, access_df, how="left", on=["curso", "groupKey", "email"])

# 3. Integrar "Forums" a la base intermedia usando curso, groupKey y email
merged_df = pd.merge(merged_df, forums_df, how="left", on=["curso", "groupKey", "email"])

# 4. Exportar el dataset consolidado
output_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/2. Para unir/merged_dataset.csv"
merged_df.to_csv(output_path, index=False)

print(f"Dataset consolidado guardado en {output_path}")

Dataset consolidado guardado en /content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/2. Para unir/merged_dataset.csv


**2. Limpieza de examenes, accesos y foros**

In [2]:
import pandas as pd

# Definir rutas de entrada y salida
input_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/2. Para unir/merged_dataset.csv"
output_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/2. Para unir/merged_cleaned_dataset.csv"

# Cargar el dataset
df = pd.read_csv(input_path)

### 1. MANEJO DE EXÁMENES

# Contar estudiantes con datos de exámenes por curso
exam_distribution = df.groupby("curso")["total_assigned_exams"].count()
total_students_per_group = df.groupby("curso")["email"].count()

# Identificar los casos de exámenes
courses_without_exams = exam_distribution[exam_distribution == 0].index  # Caso 1
courses_with_all_exams = exam_distribution[exam_distribution == total_students_per_group].index  # Caso 2
courses_some_missing_exams = exam_distribution[
    (exam_distribution / total_students_per_group >= 0.95) & (exam_distribution / total_students_per_group < 1)
].index  # Caso 3
courses_few_exams = exam_distribution[
    (exam_distribution / total_students_per_group <= 0.05) & (exam_distribution / total_students_per_group > 0)
].index  # Caso 4

# Crear la variable "no_exam_data" basado en los casos identificados
df["no_exam_data"] = 0  # Valor por defecto

df.loc[df["curso"].isin(courses_without_exams), "no_exam_data"] = 1  # Caso 1
df.loc[df["curso"].isin(courses_some_missing_exams), "no_exam_data"] = 2  # Caso 3
df.loc[df["curso"].isin(courses_few_exams), "no_exam_data"] = 3  # Caso 4

### 2. MANEJO DE ACCESOS

# Lista de variables de acceso
access_variables = [
    "total_time", "average_time", "active_days_count",
    "desktop_sessions_count", "mobile_sessions_count",
    "desktop_imputed_session_time_count", "mobile_imputed_session_time_count",
    "total_sessions_count", "total_imputed_session_time_count", "imputed_ratio"
]

# Crear la variable "missing_access_data" (1 si todas las variables de acceso están ausentes)
df["missing_access_data"] = df[access_variables].isnull().all(axis=1).astype(int)

# Contar registros sin datos de acceso
missing_access_records = df["missing_access_data"].sum()

# Contar grupos completos sin datos de acceso
missing_access_groups = df.groupby("curso")["missing_access_data"].mean()
missing_access_groups = (missing_access_groups == 1).sum()  # Contar solo grupos donde todos los registros no tienen acceso

### 3. MANEJO DE FOROS

# Lista de variables de foros
forum_variables = [
    "total_interacciones", "foros_unicos", "primer_interaccion",
    "ultima_interaccion", "rango_tiempo"
]

# Crear la variable "no_forum_data" (1 si todas las variables de foros están ausentes)
df["no_forum_data"] = df[forum_variables].isnull().all(axis=1).astype(int)

# Rellenar variables de conteo con 0 sin usar inplace=True
df["total_interacciones"] = df["total_interacciones"].fillna(0)
df["foros_unicos"] = df["foros_unicos"].fillna(0)
df["rango_tiempo"] = df["rango_tiempo"].fillna(0)

# Mantener "primer_interaccion" y "ultima_interaccion" como NaN
# df["primer_interaccion"].fillna("2000-01-01", inplace=True)  # Esta línea se eliminó para dejar NaN
# df["ultima_interaccion"].fillna("2000-01-01", inplace=True)  # Esta línea se eliminó para dejar NaN

# Contar el número total de cursos sin foros
courses_without_forums = df.groupby("curso")["no_forum_data"].mean()
courses_without_forums = (courses_without_forums == 1).sum()  # Contar solo los cursos donde todos los registros no tienen foros

### 4. IMPRESIÓN DE RESULTADOS

print("\n Distribución de Casos por Curso (Exámenes):")
print(f" Caso 1 - Cursos sin exámenes: {len(courses_without_exams)}")
print(f" Caso 2 - Cursos donde todos tienen exámenes: {len(courses_with_all_exams)}")
print(f" Caso 3 - Cursos con solo 5% alumnos sin exámenes: {len(courses_some_missing_exams)}")
print(f" Caso 4 - Cursos con solo 95% alumnos sin exámenes: {len(courses_few_exams)}")

print("\n Análisis de Datos Faltantes en Accesos:")
print(f" Total de registros individuales sin datos de acceso: {missing_access_records}")
print(f" Total de grupos (curso) donde todos los estudiantes no tienen datos de acceso: {missing_access_groups}")

print("\n Análisis de Datos Faltantes en Foros:")
print(f" Total de registros individuales sin datos de foros: {df['no_forum_data'].sum()}")
print(f" Total de cursos (curso) sin foros: {courses_without_forums}")

### 5. GUARDAR EL DATASET LIMPIO

df.to_csv(output_path, index=False)
print(f"\n Dataset limpio guardado en: {output_path}")



 Distribución de Casos por Curso (Exámenes):
 Caso 1 - Cursos sin exámenes: 23
 Caso 2 - Cursos donde todos tienen exámenes: 140
 Caso 3 - Cursos con solo 5% alumnos sin exámenes: 30
 Caso 4 - Cursos con solo 95% alumnos sin exámenes: 1

 Análisis de Datos Faltantes en Accesos:
 Total de registros individuales sin datos de acceso: 2266
 Total de grupos (curso) donde todos los estudiantes no tienen datos de acceso: 54

 Análisis de Datos Faltantes en Foros:
 Total de registros individuales sin datos de foros: 4489
 Total de cursos (curso) sin foros: 145

 Dataset limpio guardado en: /content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/2. Para unir/merged_cleaned_dataset.csv


**3. Elimina registros sin examen e imputa valores donde el faltante es 5% o menor**

In [5]:
import pandas as pd

# Definir las rutas de los archivos
input_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/2. Para unir/merged_cleaned_dataset.csv"
output_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/2. Para unir/merged_exam_cleaned_dataset.csv"

# Cargar el dataset
df = pd.read_csv(input_path)

# Filtrar registros donde no_exam_data es 1 o 3
df = df[~df["no_exam_data"].isin([1, 3])]

# Lista de variables referentes a exámenes
exam_variables = [
    "total_assigned_exams", "total_exams_finished", "total_exams_unfinished", "exam_submission_rate",
    "avg_exam_normalized_score", "avg_exam_standard_score", "max_exam_normalized_score", "min_exam_normalized_score",
    "exam_score_variability", "total_perfect_exam_scores", "total_questions_answered", "total_correct_answers",
    "total_incorrect_answers", "accuracy_rate", "total_time_spent_on_exams", "avg_exam_time", "max_exam_time",
    "min_exam_time", "outlier_exam_attempts", "general_outlier_exam_attempts", "outlier_exam_rate", "total_exam_incidents",
    "avg_exam_incidents"
]

# Procesar registros donde no_exam_data es 2 y solo imputar si son nulos
def process_exam_data(group):
    if group["no_exam_data"].iloc[0] == 2:
        group.loc[group["total_assigned_exams"].isna(), "total_assigned_exams"] = round(group["total_assigned_exams"].mean())
        group.loc[group["total_exams_unfinished"].isna(), "total_exams_unfinished"] = group["total_assigned_exams"]
        for var in exam_variables:
            if var not in ["total_assigned_exams", "total_exams_unfinished"]:
                group.loc[group[var].isna(), var] = 0
    return group

# Aplicar la función por curso sin modificar otras variables
df = df.groupby("curso", group_keys=False).apply(process_exam_data)

# Guardar el dataset limpio
df.to_csv(output_path, index=False)

print("Dataset procesado y guardado correctamente en:", output_path)


<ipython-input-5-fe4515aab7ae>:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("curso", group_keys=False).apply(process_exam_data)


Dataset procesado y guardado correctamente en: /content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/2. Para unir/merged_exam_cleaned_dataset.csv
